# RODAR Calcula taxas de suicidio que desejamos para as cidades: Por idade ajustada (total)
Por ajuste de idade e sexo (total)
Separada por sexo, fazer a idade ajustada

In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1 import make_axes_locatable

# NÃO RODAR - FASE 01: DATA PREP - Importa bases

#### Cadmun (latitude e longitude)

Tive que substituir todas as "," por "." no arquivo csv para ler corretamente o que eu queria

In [ ]:
cadmun_final= pd.read_csv('cadmun_final.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

In [ ]:
#cadmun_final.head()

In [ ]:
cadmun_final.dtypes
#MUNCOD          int64
#SUCESSOR      float64
#LATITUDE      float64
#LONGITUDE     float64
#ALTITUDE      float64
#AREA          float64

Valida se tem duplicado

In [ ]:
#print(cadmun_final.MUNCOD.count(), cadmun_final.MUNCOD.nunique() )

In [ ]:
#type(cadmun_final.LATITUDE.iloc[1000]+2)

In [ ]:
#cadmun_final.LATITUDE.iloc[1000]

#### distribuicao_populacao_idade_sexo_censo_2010: substitui todos os "," por .

In [ ]:
distribuicao_populacao_idade_sexo_censo_2010= pd.read_csv('distribuicao_populacao_idade_sexo_censo_2010.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

In [ ]:
#distribuicao_populacao_idade_sexo_censo_2010.head()

In [ ]:
#distribuicao_populacao_idade_sexo_censo_2010.dtypes

#### media_populacao_who: substitui todos os "," por .

In [ ]:
media_populacao_who= pd.read_csv('media_populacao_who.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

In [ ]:
#media_populacao_who.head()

In [ ]:
#media_populacao_who.dtypes

#### populacao: coloquei populacao na mao com o sexo "Ignorado", para bater com suicidio

In [ ]:
populacao= pd.read_csv('populacao.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

In [ ]:
#populacao.head()

In [ ]:
#populacao.dtypes

In [ ]:
#print(populacao.cod_mun.count(), populacao.cod_mun.nunique() )

5659 cadmun também é 5659

In [ ]:
#valida0 = populacao[~populacao.cod_mun.isin(cadmun_final.MUNCOD)]

In [ ]:
#valida0

In [ ]:
#valida0 = cadmun_final[~cadmun_final.MUNCOD.isin(populacao.cod_mun)]

In [ ]:
#valida0

#### rural_urbano: adicionei uma coluna com o muncod com 6 digitos ao invés de 7

In [ ]:
rural_urbano= pd.read_csv('rural_urbano.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

In [ ]:
#rural_urbano.head()

In [ ]:
#rural_urbano.dtypes

In [ ]:
#print(rural_urbano.MUNCOD.count(), rural_urbano.MUNCOD.nunique() )

Não tem a classificação para todos os municípios... cadmun tem 5.659... vamos descobrir quais são eles

In [ ]:
#valida1 = cadmun_final[~cadmun_final.MUNCOD.isin(rural_urbano.MUNCOD)]

In [ ]:
#valida1.to_csv('valida_municipio_sem_rural_urbano.csv')

#### suicidio: verifiquei sexo e tudo ok

In [ ]:
suicidio= pd.read_csv('suicidio.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

In [ ]:
#suicidio.head()

In [ ]:
#suicidio.dtypes

In [ ]:
#print(suicidio.cod_ibge_num.count(), suicidio.cod_ibge_num.nunique() )

Tem menos municipio que a cadmin... cadmun tem 5.659... vamos descobrir quais são

Primeira validação: descobrir se tem algum município da cadmun que não está aqui

In [ ]:
#valida2 = cadmun_final[~cadmun_final.MUNCOD.isin(suicidio.cod_ibge_num)]

In [ ]:
#valida2.to_csv('valida2_esta_cadmun_naoesta_suicidio.csv')

62 municípios que foram transferidos ou extintos que não estão na base de suicidio

Agora avalio quem está no suicidio e não está na cadmun

In [ ]:
#valida3 = suicidio[~suicidio.cod_ibge_num.isin(cadmun_final.MUNCOD)]

In [ ]:
#valida3.to_csv('valida3_esta_suicidio_naoesta_cadmun.csv')

Nenhum, que bom

### Cruza base a partir da população

In [ ]:
#print(populacao.cod_mun.count(), populacao.cod_mun.nunique() )

In [ ]:
#populacao[populacao.cod_mun == 355030] 

In [ ]:
#suicidio[suicidio.cod_ibge_num == 355030] 

In [ ]:
populacao_suicidio1=populacao.copy(deep=True)

In [ ]:
populacao_suicidio1=populacao_suicidio1.merge(suicidio, left_on=['cod_mun', 'ano', 'sexo'], right_on=['cod_ibge_num', 'ano', 'sexo'], how='left', suffixes=('_pop', '_suic'))  

In [ ]:
#populacao_suicidio1[populacao_suicidio1.cod_ibge_num == 355030] 

In [ ]:
#populacao_suicidio1[populacao_suicidio1.cod_ibge_num.isin([355030,317010,130356])].to_csv('valida7_cruzamento_municipios.csv')

In [ ]:
#print(populacao_suicidio1.cod_mun.count(), populacao_suicidio1.cod_mun.nunique() )

In [ ]:
populacao_suicidio1=populacao_suicidio1.fillna(0)

In [ ]:
#populacao_suicidio1[populacao_suicidio1.cod_ibge_num.isin([355030,317010,130356])].to_csv('valida8_cruzamento_municipios_fillna.csv')

Cruzou certo, validados os csvs

#### Cruza proporções

In [ ]:
#populacao_suicidio1.dtypes

In [ ]:
populacao_suicidio1=populacao_suicidio1.merge(
distribuicao_populacao_idade_sexo_censo_2010, on=['sexo'], how='left', suffixes=('', '_propcenso'))  

In [ ]:
#populacao_suicidio1.dtypes

In [ ]:
populacao_suicidio1=populacao_suicidio1.fillna(0)

In [ ]:
populacao_suicidio1_fem=populacao_suicidio1[populacao_suicidio1.sexo=='feminino']

In [ ]:
#populacao_suicidio1_fem.count()

In [ ]:
populacao_suicidio1_masc=populacao_suicidio1[populacao_suicidio1.sexo=='masculino']

In [ ]:
#populacao_suicidio1_masc.count()

In [ ]:
populacao_suicidio2 = populacao_suicidio1.groupby(['ano','cod_mun']).agg(
                        pop_total=('total_pop','sum'),
                        pop_0_a_4_anos=('0_a_4_anos_pop','sum'),
                        pop_5_a_14_anos=('5_a_14_anos_pop','sum'),
                        pop_15_a_24_anos=('15_a_24_anos_pop','sum'),
                        pop_25_a_34_anos=('25_a_34_anos_pop','sum'),
                        pop_35_a_44_anos=('35_a_44_anos_pop','sum'),
                        pop_45_a_54_anos=('45_a_54_anos_pop','sum'),
                        pop_55_a_64_anos=('55_a_64_anos_pop','sum'),
                        pop_65_a_74_anos=('65_a_74_anos_pop','sum'),
                        pop_75_anos_e_mais=('75_anos_e_mais_pop','sum'),
                        pop_idade_ignorada=('idade_ignorada_pop','sum'),
                        suic_total=('total_suic','sum'),
                        suic_0_a_4_anos=('0_a_4_anos_suic','sum'),
                        suic_5_a_14_anos=('5_a_14_anos_suic','sum'),
                        suic_15_a_24_anos=('15_a_24_anos_suic','sum'),
                        suic_25_a_34_anos=('25_a_34_anos_suic','sum'),
                        suic_35_a_44_anos=('35_a_44_anos_suic','sum'),
                        suic_45_a_54_anos=('45_a_54_anos_suic','sum'),
                        suic_55_a_64_anos=('55_a_64_anos_suic','sum'),
                        suic_65_a_74_anos=('65_a_74_anos_suic','sum'),
                        suic_75_anos_e_mais=('75_anos_e_mais_suic','sum'),
                        suic_idade_ignorada=('idade_ignorada_suic','sum')
                        ).reset_index()

#### Validações da agregação

In [ ]:
#valida1 = populacao_suicidio2.copy()

In [ ]:
#valida1['pop_valida_total'] = (-valida1.pop_total+
#valida1.pop_0_a_4_anos+
#valida1.pop_5_a_14_anos+
#valida1.pop_15_a_24_anos+
#valida1.pop_25_a_34_anos+
#valida1.pop_35_a_44_anos+
#valida1.pop_45_a_54_anos+
#valida1.pop_55_a_64_anos+
#valida1.pop_65_a_74_anos+
#valida1.pop_75_anos_e_mais+
#valida1.pop_idade_ignorada
#)


In [ ]:
#valida1.pop_valida_total.nunique()

In [ ]:
#valida1.pop_valida_total.unique()

In [ ]:
#valida1['suic_valida_total'] = (-valida1.suic_total+
#valida1.suic_0_a_4_anos+
#valida1.suic_5_a_14_anos+
#valida1.suic_15_a_24_anos+
#valida1.suic_25_a_34_anos+
#valida1.suic_35_a_44_anos+
#valida1.suic_45_a_54_anos+
#valida1.suic_55_a_64_anos+
#valida1.suic_65_a_74_anos+
#valida1.suic_75_anos_e_mais+
#valida1.suic_idade_ignorada
#)


In [ ]:
#valida1.suic_valida_total.nunique()

In [ ]:
#valida1.suic_valida_total.unique()

Valida com os arquivos csv

In [ ]:
#valida2=valida1.groupby(['ano'])['pop_total','suic_total'].sum()

In [ ]:
#valida2.to_csv('valida10_valida_totais.csv')

Tudo validado com arquivos originais

#### Coloca proporção who na tabela

In [ ]:
populacao_suicidio2['propwho_0_a_4_anos'] = media_populacao_who['0_a_4_anos'].iloc[0]
populacao_suicidio2['propwho_5_a_14_anos'] = media_populacao_who['5_a_14_anos'].iloc[0]
populacao_suicidio2['propwho_15_a_24_anos'] = media_populacao_who['15_a_24_anos'].iloc[0]
populacao_suicidio2['propwho_25_a_34_anos'] = media_populacao_who['25_a_34_anos'].iloc[0]
populacao_suicidio2['propwho_35_a_44_anos'] = media_populacao_who['35_a_44_anos'].iloc[0]
populacao_suicidio2['propwho_45_a_54_anos'] = media_populacao_who['45_a_54_anos'].iloc[0]
populacao_suicidio2['propwho_55_a_64_anos'] = media_populacao_who['55_a_64_anos'].iloc[0]
populacao_suicidio2['propwho_65_a_74_anos'] = media_populacao_who['65_a_74_anos'].iloc[0]
populacao_suicidio2['propwho_75_anos_e_mais'] = media_populacao_who['75_anos_e_mais'].iloc[0]

#### Renomeia colunas de masculino e feminino

In [ ]:
#populacao_suicidio1_masc.dtypes

In [ ]:
populacao_suicidio2_masc = populacao_suicidio1_masc.rename(
        columns={'total_pop':'pop_total_masc',
'0_a_4_anos_pop':'pop_0_a_4_anos_masc',
'5_a_14_anos_pop':'pop_5_a_14_anos_masc',
'15_a_24_anos_pop':'pop_15_a_24_anos_masc',
'25_a_34_anos_pop':'pop_25_a_34_anos_masc',
'35_a_44_anos_pop':'pop_35_a_44_anos_masc',
'45_a_54_anos_pop':'pop_45_a_54_anos_masc',
'55_a_64_anos_pop':'pop_55_a_64_anos_masc',
'65_a_74_anos_pop':'pop_65_a_74_anos_masc',
'75_anos_e_mais_pop':'pop_75_anos_e_mais_masc',
'idade_ignorada_pop':'pop_idade_ignorada_masc',
'total_suic':'suic_total_masc',
'0_a_4_anos_suic':'suic_0_a_4_anos_masc',
'5_a_14_anos_suic':'suic_5_a_14_anos_masc',
'15_a_24_anos_suic':'suic_15_a_24_anos_masc',
'25_a_34_anos_suic':'suic_25_a_34_anos_masc',
'35_a_44_anos_suic':'suic_35_a_44_anos_masc',
'45_a_54_anos_suic':'suic_45_a_54_anos_masc',
'55_a_64_anos_suic':'suic_55_a_64_anos_masc',
'65_a_74_anos_suic':'suic_65_a_74_anos_masc',
'75_anos_e_mais_suic':'suic_75_anos_e_mais_masc',
'idade_ignorada_suic':'suic_idade_ignorada_masc',
'0_a_4_anos':'prop_0_a_4_anos_masc',
'5_a_14_anos':'prop_5_a_14_anos_masc',
'15_a_24_anos':'prop_15_a_24_anos_masc',
'25_a_34_anos':'prop_25_a_34_anos_masc',
'35_a_44_anos':'prop_35_a_44_anos_masc',
'45_a_54_anos':'prop_45_a_54_anos_masc',
'55_a_64_anos':'prop_55_a_64_anos_masc',
'65_a_74_anos':'prop_65_a_74_anos_masc',
'75_anos_e_mais':'prop_75_anos_e_mais_masc'
        }    
        ,inplace=False)

In [ ]:
populacao_suicidio2_fem = populacao_suicidio1_fem.rename(
        columns={'total_pop':'pop_total_fem',
        '0_a_4_anos_pop':'pop_0_a_4_anos_fem',
        '5_a_14_anos_pop':'pop_5_a_14_anos_fem',
        '15_a_24_anos_pop':'pop_15_a_24_anos_fem',
        '25_a_34_anos_pop':'pop_25_a_34_anos_fem',
        '35_a_44_anos_pop':'pop_35_a_44_anos_fem',
        '45_a_54_anos_pop':'pop_45_a_54_anos_fem',
        '55_a_64_anos_pop':'pop_55_a_64_anos_fem',
        '65_a_74_anos_pop':'pop_65_a_74_anos_fem',
        '75_anos_e_mais_pop':'pop_75_anos_e_mais_fem',
        'idade_ignorada_pop':'pop_idade_ignorada_fem',
        'total_suic':'suic_total_fem',
        '0_a_4_anos_suic':'suic_0_a_4_anos_fem',
        '5_a_14_anos_suic':'suic_5_a_14_anos_fem',
        '15_a_24_anos_suic':'suic_15_a_24_anos_fem',
        '25_a_34_anos_suic':'suic_25_a_34_anos_fem',
        '35_a_44_anos_suic':'suic_35_a_44_anos_fem',
        '45_a_54_anos_suic':'suic_45_a_54_anos_fem',
        '55_a_64_anos_suic':'suic_55_a_64_anos_fem',
        '65_a_74_anos_suic':'suic_65_a_74_anos_fem',
        '75_anos_e_mais_suic':'suic_75_anos_e_mais_fem',
        'idade_ignorada_suic':'suic_idade_ignorada_fem',
        '0_a_4_anos':'prop_0_a_4_anos_fem',
        '5_a_14_anos':'prop_5_a_14_anos_fem',
        '15_a_24_anos':'prop_15_a_24_anos_fem',
        '25_a_34_anos':'prop_25_a_34_anos_fem',
        '35_a_44_anos':'prop_35_a_44_anos_fem',
        '45_a_54_anos':'prop_45_a_54_anos_fem',
        '55_a_64_anos':'prop_55_a_64_anos_fem',
        '65_a_74_anos':'prop_65_a_74_anos_fem',
        '75_anos_e_mais':'prop_75_anos_e_mais_fem'
        }    
        ,inplace=False)

In [ ]:
#populacao_suicidio2.count()

In [ ]:
populacao_suicidio3=populacao_suicidio2.merge(populacao_suicidio2_masc, on=['cod_mun', 'ano'],  how='left')

In [ ]:
#populacao_suicidio3.describe()

In [ ]:
#populacao_suicidio3.head(5).to_csv('valida11_ve_se_merge.csv')

In [ ]:
populacao_suicidio3=populacao_suicidio3.merge(populacao_suicidio2_fem, on=['cod_mun', 'ano'],  how='left')

In [ ]:
#populacao_suicidio3.head(5).to_csv('valida12_ve_se_merge.csv')

In [ ]:
#populacao_suicidio3.to_csv('valida13_backup.csv')

In [ ]:
#populacao_suicidio3.ano.count()

Ok, não aumentou linhas

#### Adiciona colunas das tabelas de geo etc

In [ ]:
populacao_suicidio4=populacao_suicidio3.merge(rural_urbano, left_on=['cod_mun'], right_on=['MUNCOD'], how='left') 

In [ ]:
#populacao_suicidio4.ano.count()

In [ ]:
populacao_suicidio4=populacao_suicidio4.merge(cadmun_final[['MUNCOD', 'LATITUDE','LONGITUDE','ALTITUDE','AREA','SITUACAO','UFCOD','CAPITAL','MUNNOME','MUNNOMEX']], left_on=['cod_mun'], right_on=['MUNCOD'], how='left') 

In [ ]:
#populacao_suicidio4.head(5).to_csv('valida14_qtd_campos.csv')

In [ ]:
#populacao_suicidio4.to_csv('valida15_backup.csv')

#### Taxa ajustada

In [ ]:
populacao_suicidio4['taxa_padronizada']= ((
    (populacao_suicidio4.suic_0_a_4_anos/populacao_suicidio4.pop_0_a_4_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_0_a_4_anos+
    (populacao_suicidio4.suic_5_a_14_anos/populacao_suicidio4.pop_5_a_14_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_5_a_14_anos +
    (populacao_suicidio4.suic_15_a_24_anos/populacao_suicidio4.pop_15_a_24_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_15_a_24_anos +
    (populacao_suicidio4.suic_25_a_34_anos/populacao_suicidio4.pop_25_a_34_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_25_a_34_anos +
    (populacao_suicidio4.suic_35_a_44_anos/populacao_suicidio4.pop_35_a_44_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_35_a_44_anos +
    (populacao_suicidio4.suic_45_a_54_anos/populacao_suicidio4.pop_45_a_54_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_45_a_54_anos +
    (populacao_suicidio4.suic_55_a_64_anos/populacao_suicidio4.pop_55_a_64_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_55_a_64_anos +
    (populacao_suicidio4.suic_65_a_74_anos/populacao_suicidio4.pop_65_a_74_anos).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_65_a_74_anos +
    (populacao_suicidio4.suic_75_anos_e_mais/populacao_suicidio4.pop_75_anos_e_mais).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_75_anos_e_mais
)*100000)

In [ ]:
populacao_suicidio4[['taxa_padronizada','cod_mun','MUNNOMEX','ano']].to_csv('valida18_ranking_taxa.csv')

In [ ]:
populacao_suicidio4[populacao_suicidio4.cod_mun==510779].to_csv('valida19_taxa_alta.csv')

Valida e conta parece certa, agora seria bacana validar o valida19_taxa_alta com as bases originais só para ver se bate.
510779 com 1 suicidio faixa 65 anos em 2005. Taxa deu 246,1905: cálculo bateu e valores bati com tabnet... tudo está correto.

In [ ]:
populacao_suicidio4.dtypes.to_csv('valida20_campos.csv')

In [ ]:
populacao_suicidio4['taxa_padronizada_masc']= ((
    (populacao_suicidio4.suic_0_a_4_anos_masc/populacao_suicidio4.pop_0_a_4_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_0_a_4_anos+
    (populacao_suicidio4.suic_5_a_14_anos_masc/populacao_suicidio4.pop_5_a_14_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_5_a_14_anos +
    (populacao_suicidio4.suic_15_a_24_anos_masc/populacao_suicidio4.pop_15_a_24_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_15_a_24_anos +
    (populacao_suicidio4.suic_25_a_34_anos_masc/populacao_suicidio4.pop_25_a_34_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_25_a_34_anos +
    (populacao_suicidio4.suic_35_a_44_anos_masc/populacao_suicidio4.pop_35_a_44_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_35_a_44_anos +
    (populacao_suicidio4.suic_45_a_54_anos_masc/populacao_suicidio4.pop_45_a_54_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_45_a_54_anos +
    (populacao_suicidio4.suic_55_a_64_anos_masc/populacao_suicidio4.pop_55_a_64_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_55_a_64_anos +
    (populacao_suicidio4.suic_65_a_74_anos_masc/populacao_suicidio4.pop_65_a_74_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_65_a_74_anos +
    (populacao_suicidio4.suic_75_anos_e_mais_masc/populacao_suicidio4.pop_75_anos_e_mais_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_75_anos_e_mais
)*100000)

In [ ]:
populacao_suicidio4['taxa_padronizada_fem']= ((
    (populacao_suicidio4.suic_0_a_4_anos_fem/populacao_suicidio4.pop_0_a_4_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_0_a_4_anos+
    (populacao_suicidio4.suic_5_a_14_anos_fem/populacao_suicidio4.pop_5_a_14_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_5_a_14_anos +
    (populacao_suicidio4.suic_15_a_24_anos_fem/populacao_suicidio4.pop_15_a_24_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_15_a_24_anos +
    (populacao_suicidio4.suic_25_a_34_anos_fem/populacao_suicidio4.pop_25_a_34_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_25_a_34_anos +
    (populacao_suicidio4.suic_35_a_44_anos_fem/populacao_suicidio4.pop_35_a_44_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_35_a_44_anos +
    (populacao_suicidio4.suic_45_a_54_anos_fem/populacao_suicidio4.pop_45_a_54_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_45_a_54_anos +
    (populacao_suicidio4.suic_55_a_64_anos_fem/populacao_suicidio4.pop_55_a_64_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_55_a_64_anos +
    (populacao_suicidio4.suic_65_a_74_anos_fem/populacao_suicidio4.pop_65_a_74_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_65_a_74_anos +
    (populacao_suicidio4.suic_75_anos_e_mais_fem/populacao_suicidio4.pop_75_anos_e_mais_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.propwho_75_anos_e_mais
)*100000)

In [ ]:
populacao_suicidio4['taxa_padronizada_sexo_idade']= ((
    (populacao_suicidio4.suic_0_a_4_anos_masc/populacao_suicidio4.pop_0_a_4_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_0_a_4_anos_masc+
    (populacao_suicidio4.suic_5_a_14_anos_masc/populacao_suicidio4.pop_5_a_14_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_5_a_14_anos_masc +
    (populacao_suicidio4.suic_15_a_24_anos_masc/populacao_suicidio4.pop_15_a_24_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_15_a_24_anos_masc +
    (populacao_suicidio4.suic_25_a_34_anos_masc/populacao_suicidio4.pop_25_a_34_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_25_a_34_anos_masc +
    (populacao_suicidio4.suic_35_a_44_anos_masc/populacao_suicidio4.pop_35_a_44_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_35_a_44_anos_masc +
    (populacao_suicidio4.suic_45_a_54_anos_masc/populacao_suicidio4.pop_45_a_54_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_45_a_54_anos_masc +
    (populacao_suicidio4.suic_55_a_64_anos_masc/populacao_suicidio4.pop_55_a_64_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_55_a_64_anos_masc +
    (populacao_suicidio4.suic_65_a_74_anos_masc/populacao_suicidio4.pop_65_a_74_anos_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_65_a_74_anos_masc +
    (populacao_suicidio4.suic_75_anos_e_mais_masc/populacao_suicidio4.pop_75_anos_e_mais_masc).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_75_anos_e_mais_masc+
    
    (populacao_suicidio4.suic_0_a_4_anos_fem/populacao_suicidio4.pop_0_a_4_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_0_a_4_anos_fem+
    (populacao_suicidio4.suic_5_a_14_anos_fem/populacao_suicidio4.pop_5_a_14_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_5_a_14_anos_fem +
    (populacao_suicidio4.suic_15_a_24_anos_fem/populacao_suicidio4.pop_15_a_24_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_15_a_24_anos_fem +
    (populacao_suicidio4.suic_25_a_34_anos_fem/populacao_suicidio4.pop_25_a_34_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_25_a_34_anos_fem +
    (populacao_suicidio4.suic_35_a_44_anos_fem/populacao_suicidio4.pop_35_a_44_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_35_a_44_anos_fem +
    (populacao_suicidio4.suic_45_a_54_anos_fem/populacao_suicidio4.pop_45_a_54_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_45_a_54_anos_fem +
    (populacao_suicidio4.suic_55_a_64_anos_fem/populacao_suicidio4.pop_55_a_64_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_55_a_64_anos_fem +
    (populacao_suicidio4.suic_65_a_74_anos_fem/populacao_suicidio4.pop_65_a_74_anos_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_65_a_74_anos_fem +
    (populacao_suicidio4.suic_75_anos_e_mais_fem/populacao_suicidio4.pop_75_anos_e_mais_fem).replace(np.inf,0).fillna(0)*populacao_suicidio4.prop_75_anos_e_mais_fem
)*100000)

In [ ]:
populacao_suicidio4.to_csv('base_suicidio_final.csv')

 # Análise exploratória - COMEÇA A RODAR AQUI 17/10/2021 (Etapa anterior só dá join entre bases

In [ ]:
suicidio = pd.read_csv('base_suicidio_final.csv', header = 0, low_memory=False, index_col=[0], sep=',', encoding='latin-1')
infos_mun = gpd.read_file('bcim_2016_21_11_2018.gpkg', layer='lim_municipio_a')

In [ ]:
#Como instalar geopandas:
#https://www.youtube.com/watch?v=JpgJtdZuI6s
#https://www.lfd.uci.edu/~gohlke/pythonlibs/#fiona
#shapely estava com instalação passada, tive que desinstalar pacote e instalar de novo
#Para desinstalar pacote:
#pip uninstall awscli

In [ ]:
#help(gpd)

In [ ]:
#infos_mun = gpd.read_file('bcim_2016_21_11_2018.gpkg', layer='lim_municipio_a')

In [ ]:
#infos_mun.columns

In [ ]:
#print("Municipio :"+ infos_mun.nome[2] )
#infos_mun.geometry[2]

In [ ]:
#geocodigo é o código do IBGE

In [ ]:
#infos_mun[infos_mun.geocodigo== '2104305']
#https://www.ibge.gov.br/explica/codigos-dos-municipios.php
#2104305 Godofredo Viana
#2102606 Cândido Mendes

In [ ]:
#infos_mun[infos_mun.nome== 'Cândido Mendes']

In [ ]:
#infos_mun[infos_mun.geocodigo== '2102606']

In [ ]:
#suicidio.iloc[0]

In [ ]:
#a minha cod_mun encontra o código que eu deveria encontrar
#suicidio[suicidio.cod_mun== 210430]

In [ ]:
#a minha cod_mun encontra o código que eu deveria encontrar
#suicidio[suicidio.cod_mun== 210260]

In [ ]:
#infos_mun.dtypes

### RODAR 17/10/21: Arruma mapa

In [ ]:
mapa = infos_mun.copy()
mapa = mapa[['nome','geocodigo','geometry']]
mapa['cod_mun']=mapa['geocodigo'].str[:6]
mapa.at[5552,'geocodigo']='2102606'
mapa.at[5552,'cod_mun']='210260'
mapa['geocodigo']=mapa['geocodigo'].astype(str)
mapa = mapa[mapa.geocodigo != 'None']
mapa2=mapa.copy()
mapa2['geocodigo']=mapa2['geocodigo'].astype(int)
mapa2['cod_mun']=mapa2['cod_mun'].astype(int)

In [ ]:
#só duplicou o que já estava duplicado
#print(mapa.cod_mun.count(), mapa.cod_mun.nunique() )

In [ ]:
#altero o duplicado
# código que deve ser 2102606

In [ ]:
#mapa.iloc[5552]

In [ ]:
#mapa.iloc[5552]

In [ ]:
#só alterou em mapa, está ótimo
#infos_mun.iloc[5552]

In [ ]:
#ok, agora não encontro duplicado
#print(mapa.cod_mun.count(), mapa.cod_mun.nunique() )

In [ ]:
#print(suicidio.cod_mun.count(), suicidio.cod_mun.nunique() )

In [ ]:
#suicidio.head(1).to_csv('valida21_campos_suicidio.csv')

In [ ]:
#mapa.dtypes

In [ ]:
#mapa.groupby(['geocodigo'])[['geocodigo']].count().rename(columns={'geocodigo':'qtd_codigo'}).to_csv('valida22_tipo_geo.csv')

In [ ]:

#mapa['cod_mun']=mapa['cod_mun'].astype(int)

In [ ]:
#for index,row in mapa.iterrows():
#    try:
#        int(row['geocodigo'])
#    except:
#        print('nao converteu ', row['geocodigo'])

In [ ]:
#mapa[mapa.geocodigo=='None']

In [ ]:
#Tira nulo

In [ ]:
#mapa[mapa.geocodigo=='None']

In [ ]:
#mapa2.dtypes

## Não precisa rodar: Coloca quantis em uma base agrupada por cod_mun

In [ ]:
#suicidio.head(5)

In [ ]:
#suicidio.head(5).to_csv('valida30_boxplotcidade.csv')

In [ ]:
def q25(x):
    return x.quantile(0.25)

def q50(x):
    return x.quantile(0.50)

def q75(x):
    return x.quantile(0.75)

def q90(x):
    return x.quantile(0.90)

In [ ]:
suicidio_agregado = suicidio.groupby(['cod_mun']).agg(min_taxa=('taxa_padronizada', 'min'),
                                                            max_taxa=('taxa_padronizada','max'),
                                                            media_taxa=('taxa_padronizada','mean'),
                                                            q25_taxa=('taxa_padronizada',q25),
                                                            q50_taxa=('taxa_padronizada',q50),
                                                            q75_taxa=('taxa_padronizada',q75),
                                                            q90_taxa=('taxa_padronizada',q90), 
                                                            valida_anos=('ano','count'),
                                                            valida_anos_unico=('ano','nunique'),
                                                            max_suic=('suic_total','max'),
                                                            min_suic=('suic_total','min'),
                                                            media_suic=('suic_total','mean'),
                                                            q25_suic=('suic_total',q25),
                                                            q50_suic=('suic_total',q50),
                                                            q75_suic=('suic_total',q75),
                                                            q90_suic=('suic_total',q90),
                                                            max_pop_total=('pop_total', 'max')
                                                             ).reset_index()

## NÃO PRECISA RODAR: AVALIA EM NÚMEROS O QUE SERIA O OUTLIER

In [ ]:
suicidio_agregado.to_csv('valida31_municipio.csv')

In [ ]:
suicidio_agregado.head(5)

In [ ]:
#valida se base está criada certo

In [ ]:
suicidio_agregado.loc[suicidio_agregado['valida_anos'] != suicidio_agregado['valida_anos_unico']]

In [ ]:
suicidio_agregado.cod_mun.count()

In [ ]:
suicidio_agregado.loc[suicidio_agregado['q25_taxa'] ==0].cod_mun.count()

In [ ]:
suicidio_agregado.loc[suicidio_agregado['max_taxa'] ==0].cod_mun.count()

In [ ]:
suicidio_agregado.loc[suicidio_agregado['max_taxa'] >40].cod_mun.count()

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['max_taxa'] >40) & (suicidio_agregado['q25_taxa'] !=0)].cod_mun.count()

In [ ]:
#110045

In [ ]:
suicidio_agregado.loc[suicidio_agregado['cod_mun'] ==110045]

In [ ]:
suicidio.loc[suicidio['cod_mun'] ==110045].to_csv('valida24_cidade.csv')

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['q25_taxa'] ==0) & (suicidio_agregado['q50_taxa'] !=0)].cod_mun.count()

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['q25_taxa'] ==0) & (suicidio_agregado['q50_taxa'] !=0)].sort_values(by='max_taxa', ascending=False)

In [ ]:
suicidio.loc[suicidio['cod_mun'] ==522180].to_csv('valida25_cidade.csv')

In [ ]:
suicidio.loc[suicidio['cod_mun'] ==431475][['ano','cod_mun','pop_total','suic_total','taxa_padronizada']].to_csv('valida27_cidade.csv')

In [ ]:
# quero analisar apenas cidades que tiveram mais de 20 suicídios por ano, igual artigo dos eua

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['max_suic'] >=20)].sort_values(by='max_taxa', ascending=False)

In [ ]:
suicidio.loc[suicidio['cod_mun'] ==250790][['ano','cod_mun','pop_total','suic_total','taxa_padronizada']].to_csv('valida28_cidade.csv')

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['q50_suic'] >=20)].sort_values(by='max_taxa', ascending=False)

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['q50_suic'] >=20)].sort_values(by='max_taxa', ascending=False).cod_mun.count()

In [ ]:
suicidio.loc[suicidio['cod_mun'] ==431410][['ano','cod_mun','pop_total','suic_total','taxa_padronizada']].to_csv('valida29_cidade.csv')

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['min_suic'] >=20)].sort_values(by='max_taxa', ascending=False)

In [ ]:
## max_taxa encontrada é 12.110413, valor que utilizarei para conseguir observar no mapa taxas altas

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['min_suic'] >=20)].sort_values(by='max_taxa', ascending=False).cod_mun.count()

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['min_suic'] >=20)].sort_values(by='max_taxa', ascending=False)

In [ ]:
suicidio_agregado.loc[(suicidio_agregado['q75_suic'] >=20)].sort_values(by='max_taxa', ascending=False)

In [ ]:
suicidio.loc[suicidio['cod_mun'] ==431680][['ano','cod_mun','pop_total','suic_total','taxa_padronizada']].to_csv('valida29_cidade.csv')

## RASCUNHO - APAGAR

In [ ]:
suicidio.ano.max()

In [ ]:
suicidio.ano.min()

In [ ]:
suicidio2 = pd.read_csv('base_suicidio_final.csv', header = 0, low_memory=False, index_col=[0], sep=',', encoding='latin-1')

In [ ]:
suicidio2.head()

In [ ]:
# First create some toy data:
x = np.linspace(0, 2*np.pi, 400)
y = np.sin(x**2)



In [ ]:
# Create just a figure and only one subplot
fig, ax = plt.subplots(1,1,figsize=(20,30))
ax.plot(x, y)
ax.set_title('Simple plot')

### NÃO PRECISA RODAR: Decidido o que irei fazer: só manter cidades que tem suicidio em 75% dos anos e fixar taxa em 13 (12.110413 foi o valor máximo de suicídio que encontrou na análise das cidades em todos os anos com suicidio acima de 20 unidades brutas - 20 unidades brutas por ano é o que o artigo dos EUA considera um valor bom para considerar significativo).
Fixo o valor para ver no mapa como um valor alto e conseguir comparar entre cidades.
Se o objetivo é analisar se uma cidade teve algo fora do normal ao longo do tempo, não posso mexer nessa métrica (em que eu coloco um valor máximo). Na análise temporal temos que voltar à base original então.

In [ ]:
suicidio.head()

In [ ]:
suicidio_trataoutlier = suicidio.copy()

In [ ]:
mun_considerados=suicidio_agregado.loc[suicidio_agregado['q25_taxa'] !=0][['cod_mun']]

In [ ]:
mun_considerados.count()

In [ ]:
mun_considerados.head()

In [ ]:
#isso dá 21% dos municípios sendo considerados para plotar o mapa

In [ ]:
print(suicidio_trataoutlier.cod_mun.count(), suicidio_trataoutlier.cod_mun.nunique() )

In [ ]:
suicidio_trataoutlier = suicidio_trataoutlier.loc[suicidio_trataoutlier['cod_mun'].isin(mun_considerados.cod_mun)]

In [ ]:
suicidio_trataoutlier.head()

In [ ]:
suicidio_trataoutlier=suicidio_trataoutlier.reset_index(drop=True)

In [ ]:
suicidio_trataoutlier.head()

In [ ]:
print(suicidio_trataoutlier.cod_mun.count(), suicidio_trataoutlier.cod_mun.nunique() )

In [ ]:
suicidio_trataoutlier['taxa_padronizada'] = np.where(suicidio_trataoutlier.taxa_padronizada>13,13,suicidio_trataoutlier.taxa_padronizada)

In [ ]:
suicidio_trataoutlier['taxa_padronizada_sexo_idade'] = np.where(suicidio_trataoutlier.taxa_padronizada_sexo_idade>13,13,suicidio_trataoutlier.taxa_padronizada_sexo_idade)

In [ ]:
suicidio_trataoutlier.to_csv('base_suicidio_outlier_taxa13_quantil25.csv')

In [ ]:
suicidio_trataoutlier.head()

## RODAR 17/10/21 - PEGA BASE COM A DEFINIÇÃO DE OUTLIER 01 TRATADA: no mínimo 18 anos com taxa de suicídio e todas as taxas cravadas em no máximo 13.

In [ ]:
suicidio_trataoutlier2 = pd.read_csv('base_suicidio_outlier_taxa13_quantil25.csv', header = 0, low_memory=False, index_col=[0], sep=',', encoding='latin-1')

In [ ]:
#suicidio_trataoutlier2.head()

## Função que plota mapa genérico, de acordo com as especificações feitas

In [ ]:
#Plota mapa de acordo com o que eu quero

In [ ]:
#from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
# https://geopandas.org/docs/user_guide/mapping.html ajuda a entender como editar mapas

In [ ]:
def plota_mapa(suicidio, mapa, ano, taxa):
    suicidio_ano = suicidio[suicidio.ano == ano][['cod_mun','taxa_padronizada','taxa_padronizada_sexo_idade']]
    mapa_ano=mapa.merge(suicidio_ano, on=['cod_mun'], how='left') 
    #plota com nulo
    %matplotlib inline

    fig, ax = plt.subplots(1, 1,figsize = (20,30))
    
    #tentando deixar imagem maior
    #ax=fig.add_axes([0, 0.2, 0.6, 0.6])
    
    #para arrumar legenda
    divider= make_axes_locatable(ax)
    cax = divider.append_axes("right", size="10%", pad=0.1)
    
    mapa_ano.plot(column = taxa,
                cmap ='RdYlBu_r',
                #figsize = (40,20),
                ax=ax,
                legend = True,
                #arrumar proporcao legenda
                cax=cax,
                #Deixou mapa pequeno
                #legend_kwds={'label': taxa,
                #            'orientation': 'horizontal'},
                edgecolor = 'grey',
                missing_kwds={'color':'lightgrey','label':'Missing values'}
                )
    
    #plota como zero
    #mapa_ano=mapa_ano.fillna(0)
    #%matplotlib inline
    #mapa_ano.plot(column = taxa,
    #          cmap ='RdYlBu_r',
    #          figsize = (40,20),
    #          legend = True,
    #          edgecolor = 'grey'
    #         )
    #plt.show()

In [ ]:
plota_mapa(suicidio_trataoutlier2, mapa2, 2010,'taxa_padronizada')

In [ ]:
plota_mapa(suicidio_trataoutlier2, mapa2, 2019,'taxa_padronizada')

In [ ]:
def plota_mapa_comparativo(suicidio, mapa, ano, taxa, ano2):
    suicidio_ano_1 = suicidio[suicidio.ano == ano][['cod_mun','taxa_padronizada','taxa_padronizada_sexo_idade']]
    suicidio_ano_2 = suicidio[suicidio.ano == ano2][['cod_mun','taxa_padronizada','taxa_padronizada_sexo_idade']]
    mapa_ano=mapa.merge(suicidio_ano_1, on=['cod_mun'], how='left')
    mapa_ano2=mapa.merge(suicidio_ano_2, on=['cod_mun'], how='left') 
    #plota com nulo
    %matplotlib inline

    fig, (ax1,ax2) = plt.subplots(1, 2,figsize = (20,30))
    
    ##figsize = (20,30)
    #tentando deixar imagem maior
    #ax=fig.add_axes([0, 0.2, 0.6, 0.6])
    
    #para arrumar legenda
    #divider= make_axes_locatable(ax)
    #cax = divider.append_axes("right", size="10%", pad=0.1)
    
    mapa_ano.plot(column = taxa,
                cmap ='RdYlBu_r',
                #figsize = (40,20),
                ax=ax1,
                #legend = True,
                #arrumar proporcao legenda
                #cax=cax,
                #Deixou mapa pequeno
                #legend_kwds={'label': taxa,
                #            'orientation': 'horizontal'},
                edgecolor = 'grey',
                missing_kwds={'color':'lightgrey','label':'Missing values'}
                )
    mapa_ano2.plot(column = taxa,
                cmap ='RdYlBu_r',
                #figsize = (40,20),
                ax=ax2,
                #legend = True,
                #arrumar proporcao legenda
                #cax=cax,
                #Deixou mapa pequeno
                #legend_kwds={'label': taxa,
                #            'orientation': 'horizontal'},
                edgecolor = 'grey',
                missing_kwds={'color':'lightgrey','label':'Missing values'}
                )
    
    #plota como zero
    #mapa_ano=mapa_ano.fillna(0)
    #%matplotlib inline
    #mapa_ano.plot(column = taxa,
    #          cmap ='RdYlBu_r',
    #          figsize = (40,20),
    #          legend = True,
    #          edgecolor = 'grey'
    #         )
    #plt.show()

In [ ]:
plota_mapa_comparativo(suicidio_trataoutlier2, mapa2, 2010,'taxa_padronizada',2019)

## Tentando dar zoom nas UFs para ver o que cresceu

In [ ]:
mapa2.head()

In [ ]:
#ensinando código do municíío do IBGE https://edisciplinas.usp.br/pluginfile.php/4344407/mod_assign/intro/Tabela-Codigo-de-Municipio-do-IBGE.pdf

In [ ]:
mapa2.dtypes

In [ ]:
mapa3 = mapa2.copy()

In [ ]:
mapa3['cod_uf'] = mapa3.cod_mun.astype(str).str[:2].astype(int)

In [ ]:
mapa3['cod_regiao'] = mapa3.cod_mun.astype(str).str[:1].astype(int)

In [ ]:
mapa3.head()

In [ ]:
#seleciona são paulo (olhei no site que plotei acima o código de são paulo)

In [ ]:
#faço com todas as regiões (1,2,3,4,5,6) - Copio imagens para o ppt
mapa4 = mapa3.loc[mapa3['cod_regiao'] == 5]
#mapa4 = mapa3.loc[mapa3['cod_uf'] == 31]

In [ ]:
plota_mapa(suicidio_trataoutlier2, mapa4, 2010,'taxa_padronizada')

In [ ]:
plota_mapa(suicidio_trataoutlier2, mapa4, 2019,'taxa_padronizada')

In [ ]:
plota_mapa_comparativo(suicidio_trataoutlier2, mapa4, 2010,'taxa_padronizada',2019)

In [ ]:
for i in ()

# RODAR PARA CLUSTERIZAÇÃO TEMPORAL -  FAZ VIRAR NO FORMATO NECESSÁRIO PARA CLUSTERIZAÇÃO TEMPORAL

In [ ]:
#suicidio.head()

In [ ]:
# taxa_padronizada

In [ ]:
# Evita casos absurdos

## Para a base de suicídio sem retirar nulos por 25% do tempo

In [ ]:
suicidio['taxa_padronizada'] = np.where(suicidio.taxa_padronizada>13,13,suicidio.taxa_padronizada)
suicidio2 = suicidio[['cod_mun','ano','taxa_padronizada']].sort_values(by=['cod_mun','ano'], ascending=[True,True])
teste = suicidio2.pivot(index='cod_mun', columns='ano', values='taxa_padronizada')
teste2 = teste.loc[(teste.sum(axis=1) != 0)]
teste_array=teste2.to_numpy()

In [ ]:
#suicidio2 = suicidio[['cod_mun','ano','taxa_padronizada']].sort_values(by=['cod_mun','ano'], ascending=[True,True])

In [ ]:
#suicidio2.head(48)

In [ ]:
#teste = suicidio2.pivot(index='cod_mun', columns='ano', values='taxa_padronizada')

In [ ]:
#teste.dtypes

In [ ]:
#teste.head()

In [ ]:
#print(type(teste))

In [ ]:
#tira linha que soma zero (peguei no google esse código)
#https://stackoverflow.com/questions/23573052/pandas-dataframe-how-do-i-remove-all-columns-and-rows-that-sum-to-0

In [ ]:
#teste2 = teste.loc[(teste.sum(axis=1) != 0)]

In [ ]:
#teste2

In [ ]:
#teste_array=teste2.to_numpy()

In [ ]:
teste_array

In [ ]:
len(teste_array)

## Utilizando a base que tira municípios com nulos

In [ ]:
suicidio_trataoutlier2.head()

In [ ]:
#verifica se a taxa padronizada está no máximo 13
suicidio_trataoutlier2.taxa_padronizada.max()

In [ ]:
base_interm = suicidio_trataoutlier2[['cod_mun','ano','taxa_padronizada']].sort_values(by=['cod_mun','ano'], ascending=[True,True])
base_interm2 = base_interm.pivot(index='cod_mun', columns='ano', values='taxa_padronizada')
base_interm3 = base_interm2.loc[(base_interm2.sum(axis=1) != 0)]
array_suicidio_trataoutlier=base_interm3.to_numpy()

In [ ]:
len(array_suicidio_trataoutlier)

##  INÍCIO CLUSTERIZAÇÃO TEMPORAL (PEGUEI DO ARQUIVO DA MESMA PASTA QUE ESTE - PAREI AQUI, FICOU RUIM. PRÓXIMO PASSO É FAZER A MESMA CLUSTERIZAÇÃO COM O TRATAMENTO DE OUTLIER

In [ ]:
import numpy
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [ ]:
seed = 0
numpy.random.seed(seed)

In [ ]:
#normalizo

In [ ]:
#COMPLETAR AQUI COM A BASE QUE EU QUERO!!!!!!!!!!!!!!!!!!!!!!
teste_array=array_suicidio_trataoutlier

In [ ]:
len(teste_array)

In [ ]:
teste_array = TimeSeriesScalerMeanVariance().fit_transform(teste_array)

In [ ]:
# Euclidean k-means
print("Euclidean k-means")
valores_inertia = []

for i in range(2, 15):
    km = TimeSeriesKMeans(n_clusters=i, verbose=True, random_state=seed)
    y_pred = km.fit_predict(teste_array)
    valores_inertia.append(km.inertia_)

In [ ]:
#plota o elbow
plt.plot(range(2, 15), valores_inertia, '-')
plt.axis([1, 15, 0, 30])
plt.show()

In [ ]:
#plota um cluster
plt.figure()
yi=13
for xx in teste_array[y_pred == yi]:
    #k- é a cor .2 é a grossura da linha
    plt.plot(xx.ravel(), "k-", alpha=.2)
plt.plot(km.cluster_centers_[yi].ravel(), "r-")
plt.xlim(0, 25)
plt.ylim(-4, 4)
plt.text(0.55, 0.85,'Cluster %d' % (yi),
         transform=plt.gca().transAxes)
if yi == 1:
    plt.title("Euclidean $k$-means")

In [ ]:
#plota todos os clusters
fig=plt.figure(figsize=(30,30))

#plota todos os gráficos
for yi in range(14):
    plt.subplot(5, 3, yi+1)
    for xx in teste_array[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(km.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, 24)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 0:
        plt.title("DBA $k$-means")

### Agora quero descobrir quais são as cidades que eu pontuei em alguns clusters... principalmente o 14 (que é o 13)

In [ ]:
len(teste_array[y_pred == 13])

In [ ]:
base_interm3
#1214 rows

In [ ]:
len(y_pred)

In [ ]:
len(teste_array)

In [ ]:
array_suicidio_trataoutlier[0]

In [ ]:
plt.figure()
plt.plot(array_suicidio_trataoutlier[0].ravel(), "k-", alpha=.2)

In [ ]:
y_pred[0]

In [ ]:
#plota um cluster
plt.figure()
yi=13

plt.plot(teste_array[0].ravel(), "k-", alpha=.2)
plt.plot(km.cluster_centers_[12].ravel(), "r-")
plt.xlim(0, 25)
plt.ylim(-4, 4)
plt.text(0.55, 0.85,'Cluster %d' % (12),
         transform=plt.gca().transAxes)
if yi == 1:
    plt.title("Euclidean $k$-means")

In [ ]:
base_interm3['cluster'] = y_pred.tolist()

In [ ]:
base_interm3.head()

In [ ]:
base_interm3.to_csv('valida32_cluster_aumento.csv')

# Preparando base para análise espacial no SatScan: número esperado de casos

No Stascan eles colocaram em uma das variáveis o número esperado de casos. O número esperado é dado se todas as localizações tivessem exatamente a mesma taxa de suicídio. Mais detalhes em: C:\Users\marco\Documents\20202sem\Mestrado\30 - SatScan Teste

In [ ]:
suicidio.head().to_csv('valida33_colunassuicidio.csv')

In [ ]:
#Tenho que tirar os estados aqui, isso vai alterar a conta

In [ ]:
#valida

In [ ]:
LATITUDE

In [ ]:
valida = suicidio[(suicidio.LATITUDE == 0) & (suicidio.LONGITUDE==0)]

In [ ]:
valida.to_csv('valida40_latlongzero.csv')

In [ ]:
valida.count()
#1.685 casos

In [ ]:
suicidio.count()
#135382 casos

In [ ]:
suicidio_interm = suicidio[((suicidio.LATITUDE != 0) | (suicidio.LONGITUDE != 0)) & (suicidio.LATITUDE.notnull()) & (suicidio.LONGITUDE.notnull())]

In [ ]:
suicidio_interm.count()
#133697
#ok, validada a conta

In [ ]:
suicidio_interm[suicidio_interm.cod_mun==220095][['cod_mun', 'LATITUDE','LONGITUDE']]

In [ ]:
suicidio_brasil = suicidio_interm.groupby(['ano'])['pop_total','pop_0_a_4_anos','pop_5_a_14_anos','pop_15_a_24_anos',
                                            'pop_25_a_34_anos','pop_35_a_44_anos','pop_45_a_54_anos','pop_55_a_64_anos',
                                            'pop_65_a_74_anos','pop_75_anos_e_mais','pop_idade_ignorada',
                                            'suic_total',
                                            'suic_0_a_4_anos','suic_5_a_14_anos','suic_15_a_24_anos','suic_25_a_34_anos',
                                            'suic_35_a_44_anos','suic_45_a_54_anos','suic_55_a_64_anos','suic_65_a_74_anos',
                                            'suic_75_anos_e_mais','suic_idade_ignorada',
                                            'pop_total_masc','pop_0_a_4_anos_masc',
                                            'pop_5_a_14_anos_masc','pop_15_a_24_anos_masc','pop_25_a_34_anos_masc',
                                            'pop_35_a_44_anos_masc','pop_45_a_54_anos_masc','pop_55_a_64_anos_masc',
                                            'pop_65_a_74_anos_masc','pop_75_anos_e_mais_masc','pop_idade_ignorada_masc',
                                            'suic_0_a_4_anos_masc','suic_5_a_14_anos_masc','suic_15_a_24_anos_masc',
                                            'suic_25_a_34_anos_masc','suic_35_a_44_anos_masc','suic_45_a_54_anos_masc',
                                            'suic_55_a_64_anos_masc','suic_65_a_74_anos_masc','suic_75_anos_e_mais_masc',
                                            'suic_idade_ignorada_masc','suic_total_masc',
                                            'pop_total_fem',
                                            'pop_0_a_4_anos_fem','pop_5_a_14_anos_fem','pop_15_a_24_anos_fem',
                                            'pop_25_a_34_anos_fem','pop_35_a_44_anos_fem','pop_45_a_54_anos_fem',
                                            'pop_55_a_64_anos_fem','pop_65_a_74_anos_fem','pop_75_anos_e_mais_fem',
                                            'pop_idade_ignorada_fem','suic_0_a_4_anos_fem','suic_5_a_14_anos_fem',
                                            'suic_15_a_24_anos_fem','suic_25_a_34_anos_fem','suic_35_a_44_anos_fem',
                                            'suic_45_a_54_anos_fem','suic_55_a_64_anos_fem','suic_65_a_74_anos_fem',
                                            'suic_75_anos_e_mais_fem','suic_idade_ignorada_fem','suic_total_fem'].agg('sum').reset_index()


In [ ]:
suicidio_brasil.to_csv('valida34_suicidioporanobrasil.csv')

In [ ]:
suicidio_brasil.head()

In [ ]:
#Cria as taxas esperadas no Brasil por ano
suicidio_brasil['taxa_esperada_total'] = suicidio_brasil['suic_total']/suicidio_brasil['pop_total']
suicidio_brasil['taxa_esperada_0_a_4_anos'] = suicidio_brasil['suic_0_a_4_anos']/suicidio_brasil['pop_0_a_4_anos']
suicidio_brasil['taxa_esperada_5_a_14_anos'] = suicidio_brasil['suic_5_a_14_anos']/suicidio_brasil['pop_5_a_14_anos']
suicidio_brasil['taxa_esperada_15_a_24_anos'] = suicidio_brasil['suic_15_a_24_anos']/suicidio_brasil['pop_15_a_24_anos']
suicidio_brasil['taxa_esperada_25_a_34_anos'] = suicidio_brasil['suic_25_a_34_anos']/suicidio_brasil['pop_25_a_34_anos']
suicidio_brasil['taxa_esperada_35_a_44_anos'] = suicidio_brasil['suic_35_a_44_anos']/suicidio_brasil['pop_35_a_44_anos']
suicidio_brasil['taxa_esperada_45_a_54_anos'] = suicidio_brasil['suic_45_a_54_anos']/suicidio_brasil['pop_45_a_54_anos']
suicidio_brasil['taxa_esperada_55_a_64_anos'] = suicidio_brasil['suic_55_a_64_anos']/suicidio_brasil['pop_55_a_64_anos']
suicidio_brasil['taxa_esperada_65_a_74_anos'] = suicidio_brasil['suic_65_a_74_anos']/suicidio_brasil['pop_65_a_74_anos']
suicidio_brasil['taxa_esperada_75_anos_e_mais'] = suicidio_brasil['suic_75_anos_e_mais']/suicidio_brasil['pop_75_anos_e_mais']
suicidio_brasil['taxa_esperada_idade_ignorada'] = suicidio_brasil['suic_idade_ignorada']/suicidio_brasil['pop_idade_ignorada']
suicidio_brasil['taxa_esperada_total_masc'] = suicidio_brasil['suic_total_masc']/suicidio_brasil['pop_total_masc']
suicidio_brasil['taxa_esperada_0_a_4_anos_masc'] = suicidio_brasil['suic_0_a_4_anos_masc']/suicidio_brasil['pop_0_a_4_anos_masc']
suicidio_brasil['taxa_esperada_5_a_14_anos_masc'] = suicidio_brasil['suic_5_a_14_anos_masc']/suicidio_brasil['pop_5_a_14_anos_masc']
suicidio_brasil['taxa_esperada_15_a_24_anos_masc'] = suicidio_brasil['suic_15_a_24_anos_masc']/suicidio_brasil['pop_15_a_24_anos_masc']
suicidio_brasil['taxa_esperada_25_a_34_anos_masc'] = suicidio_brasil['suic_25_a_34_anos_masc']/suicidio_brasil['pop_25_a_34_anos_masc']
suicidio_brasil['taxa_esperada_35_a_44_anos_masc'] = suicidio_brasil['suic_35_a_44_anos_masc']/suicidio_brasil['pop_35_a_44_anos_masc']
suicidio_brasil['taxa_esperada_45_a_54_anos_masc'] = suicidio_brasil['suic_45_a_54_anos_masc']/suicidio_brasil['pop_45_a_54_anos_masc']
suicidio_brasil['taxa_esperada_55_a_64_anos_masc'] = suicidio_brasil['suic_55_a_64_anos_masc']/suicidio_brasil['pop_55_a_64_anos_masc']
suicidio_brasil['taxa_esperada_65_a_74_anos_masc'] = suicidio_brasil['suic_65_a_74_anos_masc']/suicidio_brasil['pop_65_a_74_anos_masc']
suicidio_brasil['taxa_esperada_75_anos_e_mais_masc'] = suicidio_brasil['suic_75_anos_e_mais_masc']/suicidio_brasil['pop_75_anos_e_mais_masc']
suicidio_brasil['taxa_esperada_idade_ignorada_masc'] = suicidio_brasil['suic_idade_ignorada_masc']/suicidio_brasil['pop_idade_ignorada_masc']
suicidio_brasil['taxa_esperada_total_fem'] = suicidio_brasil['suic_total_fem']/suicidio_brasil['pop_total_fem']
suicidio_brasil['taxa_esperada_0_a_4_anos_fem'] = suicidio_brasil['suic_0_a_4_anos_fem']/suicidio_brasil['pop_0_a_4_anos_fem']
suicidio_brasil['taxa_esperada_5_a_14_anos_fem'] = suicidio_brasil['suic_5_a_14_anos_fem']/suicidio_brasil['pop_5_a_14_anos_fem']
suicidio_brasil['taxa_esperada_15_a_24_anos_fem'] = suicidio_brasil['suic_15_a_24_anos_fem']/suicidio_brasil['pop_15_a_24_anos_fem']
suicidio_brasil['taxa_esperada_25_a_34_anos_fem'] = suicidio_brasil['suic_25_a_34_anos_fem']/suicidio_brasil['pop_25_a_34_anos_fem']
suicidio_brasil['taxa_esperada_35_a_44_anos_fem'] = suicidio_brasil['suic_35_a_44_anos_fem']/suicidio_brasil['pop_35_a_44_anos_fem']
suicidio_brasil['taxa_esperada_45_a_54_anos_fem'] = suicidio_brasil['suic_45_a_54_anos_fem']/suicidio_brasil['pop_45_a_54_anos_fem']
suicidio_brasil['taxa_esperada_55_a_64_anos_fem'] = suicidio_brasil['suic_55_a_64_anos_fem']/suicidio_brasil['pop_55_a_64_anos_fem']
suicidio_brasil['taxa_esperada_65_a_74_anos_fem'] = suicidio_brasil['suic_65_a_74_anos_fem']/suicidio_brasil['pop_65_a_74_anos_fem']
suicidio_brasil['taxa_esperada_75_anos_e_mais_fem'] = suicidio_brasil['suic_75_anos_e_mais_fem']/suicidio_brasil['pop_75_anos_e_mais_fem']
suicidio_brasil['taxa_esperada_idade_ignorada_fem'] = suicidio_brasil['suic_idade_ignorada_fem']/suicidio_brasil['pop_idade_ignorada_fem']




In [ ]:
suicidio_brasil.to_csv('valida35_taxasesperadas.csv')

In [ ]:
suicidio_brasil = suicidio_brasil[['ano','taxa_esperada_total','taxa_esperada_0_a_4_anos','taxa_esperada_5_a_14_anos',
                                   'taxa_esperada_15_a_24_anos','taxa_esperada_25_a_34_anos','taxa_esperada_35_a_44_anos',
                                   'taxa_esperada_45_a_54_anos','taxa_esperada_55_a_64_anos','taxa_esperada_65_a_74_anos',
                                   'taxa_esperada_75_anos_e_mais','taxa_esperada_idade_ignorada','taxa_esperada_total_masc',
                                   'taxa_esperada_0_a_4_anos_masc','taxa_esperada_5_a_14_anos_masc','taxa_esperada_15_a_24_anos_masc',
                                   'taxa_esperada_25_a_34_anos_masc','taxa_esperada_35_a_44_anos_masc','taxa_esperada_45_a_54_anos_masc',
                                   'taxa_esperada_55_a_64_anos_masc','taxa_esperada_65_a_74_anos_masc','taxa_esperada_75_anos_e_mais_masc',
                                   'taxa_esperada_idade_ignorada_masc','taxa_esperada_total_fem','taxa_esperada_0_a_4_anos_fem',
                                   'taxa_esperada_5_a_14_anos_fem','taxa_esperada_15_a_24_anos_fem','taxa_esperada_25_a_34_anos_fem',
                                   'taxa_esperada_35_a_44_anos_fem','taxa_esperada_45_a_54_anos_fem','taxa_esperada_55_a_64_anos_fem',
                                   'taxa_esperada_65_a_74_anos_fem','taxa_esperada_75_anos_e_mais_fem',
                                   'taxa_esperada_idade_ignorada_fem']]

In [ ]:
#suicidio_brasil.to_csv('valida36_suicidiobrasil.csv')

In [ ]:
#só usa base que possui lat e long
suicidio_intermediario=suicidio_interm.merge(suicidio_brasil, on=['ano'],  how='left')

### Valida

In [ ]:
suicidio_intermediario[suicidio_intermediario.ano==2019].cod_mun.count()

In [ ]:
suicidio_intermediario[suicidio_intermediario.ano==2019].cod_mun.nunique()

In [ ]:
#suicidio_intermediario.head().to_csv('valida37_junta.csv')

In [ ]:
#faz o cálculo da quantidade de suicidios esperada
suicidio_intermediario['suic_esp_total'] = (suicidio_intermediario['pop_total']*suicidio_intermediario['taxa_esperada_total'])
suicidio_intermediario['suic_esp_0_a_4_anos'] = (suicidio_intermediario['pop_0_a_4_anos']*suicidio_intermediario['taxa_esperada_0_a_4_anos'])
suicidio_intermediario['suic_esp_5_a_14_anos'] = (suicidio_intermediario['pop_5_a_14_anos']*suicidio_intermediario['taxa_esperada_5_a_14_anos'])
suicidio_intermediario['suic_esp_15_a_24_anos'] = (suicidio_intermediario['pop_15_a_24_anos']*suicidio_intermediario['taxa_esperada_15_a_24_anos'])
suicidio_intermediario['suic_esp_25_a_34_anos'] = (suicidio_intermediario['pop_25_a_34_anos']*suicidio_intermediario['taxa_esperada_25_a_34_anos'])
suicidio_intermediario['suic_esp_35_a_44_anos'] = (suicidio_intermediario['pop_35_a_44_anos']*suicidio_intermediario['taxa_esperada_35_a_44_anos'])
suicidio_intermediario['suic_esp_45_a_54_anos'] = (suicidio_intermediario['pop_45_a_54_anos']*suicidio_intermediario['taxa_esperada_45_a_54_anos'])
suicidio_intermediario['suic_esp_55_a_64_anos'] = (suicidio_intermediario['pop_55_a_64_anos']*suicidio_intermediario['taxa_esperada_55_a_64_anos'])
suicidio_intermediario['suic_esp_65_a_74_anos'] = (suicidio_intermediario['pop_65_a_74_anos']*suicidio_intermediario['taxa_esperada_65_a_74_anos'])
suicidio_intermediario['suic_esp_75_anos_e_mais'] = (suicidio_intermediario['pop_75_anos_e_mais']*suicidio_intermediario['taxa_esperada_75_anos_e_mais'])
suicidio_intermediario['suic_esp_idade_ignorada'] = (suicidio_intermediario['pop_idade_ignorada']*suicidio_intermediario['taxa_esperada_idade_ignorada'])
suicidio_intermediario['suic_esp_total_masc'] = (suicidio_intermediario['pop_total_masc']*suicidio_intermediario['taxa_esperada_total_masc'])
suicidio_intermediario['suic_esp_0_a_4_anos_masc'] = (suicidio_intermediario['pop_0_a_4_anos_masc']*suicidio_intermediario['taxa_esperada_0_a_4_anos_masc'])
suicidio_intermediario['suic_esp_5_a_14_anos_masc'] = (suicidio_intermediario['pop_5_a_14_anos_masc']*suicidio_intermediario['taxa_esperada_5_a_14_anos_masc'])
suicidio_intermediario['suic_esp_15_a_24_anos_masc'] = (suicidio_intermediario['pop_15_a_24_anos_masc']*suicidio_intermediario['taxa_esperada_15_a_24_anos_masc'])
suicidio_intermediario['suic_esp_25_a_34_anos_masc'] = (suicidio_intermediario['pop_25_a_34_anos_masc']*suicidio_intermediario['taxa_esperada_25_a_34_anos_masc'])
suicidio_intermediario['suic_esp_35_a_44_anos_masc'] = (suicidio_intermediario['pop_35_a_44_anos_masc']*suicidio_intermediario['taxa_esperada_35_a_44_anos_masc'])
suicidio_intermediario['suic_esp_45_a_54_anos_masc'] = (suicidio_intermediario['pop_45_a_54_anos_masc']*suicidio_intermediario['taxa_esperada_45_a_54_anos_masc'])
suicidio_intermediario['suic_esp_55_a_64_anos_masc'] = (suicidio_intermediario['pop_55_a_64_anos_masc']*suicidio_intermediario['taxa_esperada_55_a_64_anos_masc'])
suicidio_intermediario['suic_esp_65_a_74_anos_masc'] = (suicidio_intermediario['pop_65_a_74_anos_masc']*suicidio_intermediario['taxa_esperada_65_a_74_anos_masc'])
suicidio_intermediario['suic_esp_75_anos_e_mais_masc'] = (suicidio_intermediario['pop_75_anos_e_mais_masc']*suicidio_intermediario['taxa_esperada_75_anos_e_mais_masc'])
suicidio_intermediario['suic_esp_idade_ignorada_masc'] = (suicidio_intermediario['pop_idade_ignorada_masc']*suicidio_intermediario['taxa_esperada_idade_ignorada_masc'])
suicidio_intermediario['suic_esp_total_fem'] = (suicidio_intermediario['pop_total_fem']*suicidio_intermediario['taxa_esperada_total_fem'])
suicidio_intermediario['suic_esp_0_a_4_anos_fem'] = (suicidio_intermediario['pop_0_a_4_anos_fem']*suicidio_intermediario['taxa_esperada_0_a_4_anos_fem'])
suicidio_intermediario['suic_esp_5_a_14_anos_fem'] = (suicidio_intermediario['pop_5_a_14_anos_fem']*suicidio_intermediario['taxa_esperada_5_a_14_anos_fem'])
suicidio_intermediario['suic_esp_15_a_24_anos_fem'] = (suicidio_intermediario['pop_15_a_24_anos_fem']*suicidio_intermediario['taxa_esperada_15_a_24_anos_fem'])
suicidio_intermediario['suic_esp_25_a_34_anos_fem'] = (suicidio_intermediario['pop_25_a_34_anos_fem']*suicidio_intermediario['taxa_esperada_25_a_34_anos_fem'])
suicidio_intermediario['suic_esp_35_a_44_anos_fem'] = (suicidio_intermediario['pop_35_a_44_anos_fem']*suicidio_intermediario['taxa_esperada_35_a_44_anos_fem'])
suicidio_intermediario['suic_esp_45_a_54_anos_fem'] = (suicidio_intermediario['pop_45_a_54_anos_fem']*suicidio_intermediario['taxa_esperada_45_a_54_anos_fem'])
suicidio_intermediario['suic_esp_55_a_64_anos_fem'] = (suicidio_intermediario['pop_55_a_64_anos_fem']*suicidio_intermediario['taxa_esperada_55_a_64_anos_fem'])
suicidio_intermediario['suic_esp_65_a_74_anos_fem'] = (suicidio_intermediario['pop_65_a_74_anos_fem']*suicidio_intermediario['taxa_esperada_65_a_74_anos_fem'])
suicidio_intermediario['suic_esp_75_anos_e_mais_fem'] = (suicidio_intermediario['pop_75_anos_e_mais_fem']*suicidio_intermediario['taxa_esperada_75_anos_e_mais_fem'])
suicidio_intermediario['suic_esp_idade_ignorada_fem'] = (suicidio_intermediario['pop_idade_ignorada_fem']*suicidio_intermediario['taxa_esperada_idade_ignorada_fem'])


In [ ]:
suicidio_intermediario

In [ ]:
#validacao
#suicidio_intermediario[suicidio_intermediario.suic_idade_ignorada!= (suicidio_intermediario.suic_idade_ignorada_masc+suicidio_intermediario.suic_idade_ignorada_fem)][['cod_mun','suic_idade_ignorada_masc','suic_idade_ignorada_fem','suic_idade_ignorada']]

In [ ]:
#excluindo casos que não reporta sexo ou idade
suicidio_intermediario['suicidio_observado_sexo_idade'] = (suicidio_intermediario['suic_0_a_4_anos_masc']+
suicidio_intermediario['suic_5_a_14_anos_masc']+suicidio_intermediario['suic_15_a_24_anos_masc']+
suicidio_intermediario['suic_25_a_34_anos_masc']+suicidio_intermediario['suic_35_a_44_anos_masc']+ suicidio_intermediario['suic_45_a_54_anos_masc']+
                                                           suicidio_intermediario['suic_55_a_64_anos_masc']+
                                                           suicidio_intermediario['suic_65_a_74_anos_masc']+
                                                           suicidio_intermediario['suic_75_anos_e_mais_masc']+
                                                           suicidio_intermediario['suic_0_a_4_anos_fem']+
                                                           suicidio_intermediario['suic_5_a_14_anos_fem']+
                                                           suicidio_intermediario['suic_15_a_24_anos_fem']+
                                                           suicidio_intermediario['suic_25_a_34_anos_fem']+
                                                           suicidio_intermediario['suic_35_a_44_anos_fem']+
                                                           suicidio_intermediario['suic_45_a_54_anos_fem']+
                                                           suicidio_intermediario['suic_55_a_64_anos_fem']+
                                                           suicidio_intermediario['suic_65_a_74_anos_fem']+
                                                           suicidio_intermediario['suic_75_anos_e_mais_fem'])



In [ ]:
#excluindo casos que não reporta sexo ou idade
suicidio_intermediario['suicidio_esperado_sexo_idade'] = (suicidio_intermediario['suic_esp_0_a_4_anos_masc']+
                                                          suicidio_intermediario['suic_esp_5_a_14_anos_masc']+
                                                          suicidio_intermediario['suic_esp_15_a_24_anos_masc']+
                                                          suicidio_intermediario['suic_esp_25_a_34_anos_masc']+
                                                          suicidio_intermediario['suic_esp_35_a_44_anos_masc']+
                                                          suicidio_intermediario['suic_esp_45_a_54_anos_masc']+
                                                          suicidio_intermediario['suic_esp_55_a_64_anos_masc']+
                                                          suicidio_intermediario['suic_esp_65_a_74_anos_masc']+
                                                          suicidio_intermediario['suic_esp_75_anos_e_mais_masc']+
                                                          suicidio_intermediario['suic_esp_0_a_4_anos_fem']+
                                                          suicidio_intermediario['suic_esp_5_a_14_anos_fem']+
                                                          suicidio_intermediario['suic_esp_15_a_24_anos_fem']+
                                                          suicidio_intermediario['suic_esp_25_a_34_anos_fem']+
                                                          suicidio_intermediario['suic_esp_35_a_44_anos_fem']+
                                                          suicidio_intermediario['suic_esp_45_a_54_anos_fem']+
                                                          suicidio_intermediario['suic_esp_55_a_64_anos_fem']+
                                                          suicidio_intermediario['suic_esp_65_a_74_anos_fem']+
                                                          suicidio_intermediario['suic_esp_75_anos_e_mais_fem'])



In [ ]:
suicidio_intermediario.suicidio_esperado_sexo_idade.sum()

In [ ]:
suicidio_intermediario.suicidio_observado_sexo_idade.sum()

In [ ]:
suicidio_intermediario.to_csv('base_suicidio_satscan.csv')

### Valida um caso - validação ok

In [ ]:
suicidio_intermediario[suicidio_intermediario.ano==2019].sort_values(by='taxa_esperada_total', ascending=False).head(5).to_csv('valida41_validacaso.csv')

In [ ]:
suicidio_intermediario[suicidio_intermediario.ano==2019].sort_values(by='suic_total', ascending=False).head(5).to_csv('valida42_validacaso.csv')

# RODAR AQUI 28/11/2021: PREPARA BASE PARA SATSCAN

In [ ]:
suicidio_intermediario = pd.read_csv('base_suicidio_satscan.csv', header = 0, low_memory=False, index_col=[0], sep=',', encoding='latin-1')

In [ ]:
#print(suicidio_intermediario.suicidio_esperado_sexo_idade.sum(),suicidio_intermediario.suicidio_observado_sexo_idade.sum() )

In [ ]:
#suicidio_intermediario.head(5)

In [ ]:
#suicidio_intermediario.head(5).to_csv('valida43_base_com_covariavei.csv')

In [ ]:
#entender quais são os tipos de lugar
#suicidio_intermediario.groupby(['ano','TIPO'])['cod_mun'].agg('count')

In [ ]:
#suicidio_intermediario.groupby(['ano'])['cod_mun'].agg('count')

In [ ]:
#satscan_geo = suicidio_intermediario[suicidio_intermediario.ano == 2019][['cod_mun','LATITUDE','LONGITUDE']] 

In [ ]:
#valida
#print(satscan_geo.cod_mun.count(), satscan_geo.cod_mun.nunique() )

In [ ]:
#satscan_geo.to_csv('satscan_geo.csv', index=False)

In [ ]:
#satscan_observado = suicidio_intermediario[suicidio_intermediario.ano == 2019][['cod_mun','suicidio_observado_sexo_idade']] 

In [ ]:
#valida
#print(satscan_observado.cod_mun.count(), satscan_observado.cod_mun.nunique() )

In [ ]:
#satscan_observado.to_csv('satscan_observado.csv', index=False)

In [ ]:
#satscan_esperado = suicidio_intermediario[suicidio_intermediario.ano == 2019][['cod_mun','suicidio_esperado_sexo_idade']] 

In [ ]:
#valida
#print(satscan_esperado.cod_mun.count(), satscan_esperado.cod_mun.nunique() )

In [ ]:
#satscan_esperado.to_csv('satscan_esperado.csv', index=False)

In [ ]:
#LEMBRAR: cada exportação eu mexi manualmente no arquivo: troquei o "." pelo @@, 
#depois dividi em colunas, depois exclui a primeira coluna. Depois voltei o "@@" para ","

## ATUALIZAÇÃO 16/01/2022: ACHO que arrumei pelos códigos mesmo, então é só colocar na ferramenta

### Ruralidade: tratar a base para ruralidade - há 4 municípios nulos

In [ ]:
#suicidio_rural_nulo = suicidio_intermediario[(suicidio_intermediario.TIPO.isnull())]

In [ ]:
#cidades do litoral de santa catarina, interior do PA, MS e RS
#suicidio_rural_nulo[['cod_mun','ano','suicidio_observado_sexo_idade','MUNNOMEX', 'TIPO']].to_csv('valida44_ruralidade_nulo.csv')

In [ ]:
#suicidio_interm_ruralidade = suicidio_intermediario.copy()

In [ ]:
#foram rural adjacente por estarem próximos de cidades grandes, mas estão no interior - e é o que é a maioria dos municípios
#suicidio_interm_ruralidade['TIPO'] = np.where(suicidio_interm_ruralidade.TIPO.isnull(),'RuralAdjacente',suicidio_interm_ruralidade.TIPO)

In [ ]:
#suicidio_rural_nulo = suicidio_interm_ruralidade[(suicidio_interm_ruralidade.TIPO.isnull())]

In [ ]:
#suicidio_rural_nulo.head(5)

In [ ]:
#suicidio_interm_ruralidade.groupby(['ano','TIPO'])['cod_mun'].agg('count')

In [ ]:
#satscan_observado_ruralidade = suicidio_interm_ruralidade[suicidio_interm_ruralidade.ano == 2019][['cod_mun','suicidio_observado_sexo_idade', 'TIPO']]

In [ ]:
#satscan_observado_ruralidade.to_csv('satscan_observado_ruralidade.csv', index=False)

In [ ]:
#satscan_esperado_ruralidade = suicidio_interm_ruralidade[suicidio_interm_ruralidade.ano == 2019][['cod_mun','suicidio_esperado_sexo_idade', 'TIPO']]

In [ ]:
#satscan_esperado_ruralidade.to_csv('satscan_esperado_ruralidade.csv', index=False)

In [ ]:
# TIVE QUE TRANSFORMAR O FLOAT DO OBSERVADO EM INT NA MÃO NO ARQUIVO

### Prepara a base espaço-temporal

In [ ]:
#Deu erro porque tem cidade que não aparece todos os anos, pensar se vou restringir data ou se vou restringir cidade.
#Posso fazer os dois: data e cidade
#abaixo começam os estudos sobre isso

In [ ]:
#satscan_observado_temporal.head(5)

In [ ]:
#tenta ver se tem algum ano que está faltando o cod_mun
#satscan_observado_temporal.groupby(['ano'])['cod_mun'].agg('count').to_csv('valida46_mun_nulo.csv')

In [ ]:
# a partir de 2013 some 1 município, por isso que a extração de geolocalização é de 2012

In [ ]:
# agora entende se há municípios novos ao longo do tempo, ou eles aparecem desde 1996, mas estão vazios mesmo

In [ ]:
#teste = satscan_observado_temporal[satscan_observado_temporal.ano==1996][['cod_mun']]

In [ ]:
#teste2 = satscan_observado_temporal[~satscan_observado_temporal.cod_mun.isin(teste.cod_mun)][['ano','cod_mun']]

In [ ]:
#teste2

In [ ]:
# cidades de 1996 estão em todos os anos no observado, acho que equalizei municípios mesmo
# sei que a partir de 2013, uma cidade some, vamos descobrir qual

In [ ]:
#teste = satscan_observado_temporal[satscan_observado_temporal.ano==2013][['cod_mun']]


In [ ]:
#teste2 = satscan_observado_temporal[~satscan_observado_temporal.cod_mun.isin(teste.cod_mun)][['ano','cod_mun']]

In [ ]:
#teste2

In [ ]:
# A cidade 431453 que sumiu
#Entre 2001 e 2002, n˜ao foram criados nem destru´ıdos munic´ıpios. Entre 2002
e 2003, n˜ao foram criados novos munic´ıpios. Por outro lado, o munic´ıpio de Pinto
Bandeira, c´odigo IBGE 431453, que tinha 2.673 habitantes em 2002, foi exclu´ıdo.
Pinto Bandeira era um distrito do munic´ıpio de Bento Gon¸calves no estado do Rio
Grande do Sul. Foi elevado `a categoria de munic´ıpio em 2001, mas extinto por
decis˜ao do Supremo Tribunal Federal em 2002.
# Faz uma análise de tudo que entra e sai ao longo do tempo
# http://jaguar.fcav.unesp.br/RME/fasciculos/v29/v29_n1/A10_Alexandre.pdf

In [ ]:
#teste

In [ ]:
#satscan_esperado_temporal.head(5)

In [ ]:
#suicidio_intermediario[suicidio_intermediario.cod_mun == 150475].to_csv('valida47_populacaonula.csv')

In [ ]:
# entender municípios que não existiam em algum ano
#suicidio_intermediario[(suicidio_intermediario.pop_total == 0) | (suicidio_intermediario.suicidio_esperado_sexo_idade == 0)][['ano','cod_mun','pop_total','suicidio_esperado_sexo_idade']].to_csv('valida48_populacaonula.csv')

#### Resultado do estudo: Dos 5567 municípios totais, 593 não aparecem em um ou mais anos. 533 não estão na base de 1996, 53 não estão na base entre 1996 e 200, 1 não está na base entre 1996 e 2008 e 6 não está não base entre 1996 e 2012.

In [ ]:
#na planilha valida 48 faço toda a análise de quem são esses municipios e crio a base excluidos_expaco_temporal que retiro os 60 casos

In [ ]:
#tirei o index_col
excluidos_espaco_temporal = pd.read_csv('excluidos_espaco_temporal.csv', header = 0, low_memory=False, sep=',', encoding='latin-1')

In [ ]:
#excluidos_espaco_temporal.head(5)

In [ ]:
#excluidos_espaco_temporal.cod_mun.count()

In [ ]:
# Vou retirar da base os excluidos para análise temporal

In [ ]:
suicidio_intermediario_munexcluidos=suicidio_intermediario[~suicidio_intermediario.cod_mun.isin(excluidos_espaco_temporal.cod_mun)]

In [ ]:
#valida

In [ ]:
#print(suicidio_intermediario.cod_mun.count(), suicidio_intermediario.cod_mun.nunique() )
#print(suicidio_intermediario_munexcluidos.cod_mun.count(), suicidio_intermediario_munexcluidos.cod_mun.nunique() )

In [ ]:
#começa a exportar para análise espaço-temporal - Lembrando que a análise vai começar em 1997

In [ ]:
suicidio_intermediario_munexcluidos = suicidio_intermediario_munexcluidos[suicidio_intermediario_munexcluidos.ano > 1996]

In [ ]:
#print(suicidio_intermediario_munexcluidos.cod_mun.count(), suicidio_intermediario_munexcluidos.cod_mun.nunique() )

In [ ]:
#de 2012 para 2013 sumiu um codmun, por isso vou pegar do ano de 2012
#vamos entender qual é esse codmun?
#testar código df[~df.country.isin(countries_to_keep)]

In [ ]:
#satscan_geo_temporal = suicidio_intermediario_munexcluidos[suicidio_intermediario_munexcluidos.ano == 1997][['cod_mun','LATITUDE','LONGITUDE']] 

In [ ]:
#satscan_geo_temporal.to_csv('satscan_geo_temporal.csv', index=False)

In [ ]:
#suicidio_intermediario_munexcluidos.suicidio_observado_sexo_idade = suicidio_intermediario_munexcluidos.suicidio_observado_sexo_idade.astype(int)

In [ ]:
 # temporal
#satscan_observado_temporal = suicidio_intermediario_munexcluidos[['ano','cod_mun','suicidio_observado_sexo_idade']] 

In [ ]:
#satscan_observado_temporal.to_csv('satscan_observado_temporal.csv', index=False)

In [ ]:
 #satscan_esperado_temporal = suicidio_intermediario_munexcluidos[['ano','cod_mun','suicidio_esperado_sexo_idade']] 

In [ ]:
#satscan_esperado_temporal.to_csv('satscan_esperado_temporal.csv', index=False)

In [ ]:
#LEMBRAR: cada exportação eu mexi manualmente no arquivo: troquei o "." pelo @@, 
#depois dividi em colunas, depois exclui a primeira coluna. Depois voltei o "@@" para ","

## ATUALIZAÇÃO 16/01/2022: ACHO que arrumei pelos códigos mesmo, então é só colocar na ferramenta

In [ ]:
#VALIDA - VI AQUI E TUDO CERTO - todos estão de 1997 a 2019 com 5.507 municipios

In [ ]:
#satscan_esperado_temporal.groupby(['ano'])['cod_mun'].agg('count')

In [ ]:
#satscan_observado_temporal.groupby(['ano'])['cod_mun'].agg('count')

In [ ]:
#satscan_geo_temporal.cod_mun.count()

# 07/03/2022: análise de joinpoint, usa codigos rodados acima

In [ ]:
suicidio_intermediario_munexcluidos.head(1).to_csv('valida50_campos.csv')

In [ ]:
joinpoint = suicidio_intermediario_munexcluidos[['cod_mun', 'MUNNOMEX','ano','taxa_padronizada_sexo_idade']].sort_values(by=['cod_mun','ano'], ascending=True)

In [ ]:
joinpoint.head(5)

In [ ]:
joinpoint.taxa_padronizada_sexo_idade.min()

In [ ]:
joinpoint[joinpoint.taxa_padronizada_sexo_idade != 0].taxa_padronizada_sexo_idade.min()

In [ ]:
#deu problema para taxas zeradas no joinpoint, vou tirar as taxas zeradas

In [ ]:
joinpoint['taxa_padronizada_sexo_idade'] = np.where(joinpoint.taxa_padronizada_sexo_idade==0,0.001,joinpoint.taxa_padronizada_sexo_idade)

In [ ]:
joinpoint.taxa_padronizada_sexo_idade.min()

In [ ]:
joinpoint.to_csv('base_joinpoint.csv', index=False)

# 12/03/2022: PREPARA UMA BASE COM TODAS AS SAÍDAS PARA FAZER OS ESTUDOS DE RURALIDADE E O DASHBOARD GERAL

In [ ]:
#cadmun_final= pd.read_csv('cadmun_final.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

#suicidio_brasil

#suicidio_intermediario=suicidio_interm.merge(suicidio_brasil, on=['ano'],  how='left')

In [ ]:
#TRANSFORMAR OS ARQUIVOS EM CSV COM VÍRGULA EM PONTO, ASSIM O PYTHON ENTENDE CERTINHO O TIPO
base_espaco_consolidada = pd.read_csv('base_espaco_consolidada.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')
base_espaco_tempo_consolidada = pd.read_csv('base_espaco_tempo_consolidada.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')
base_joinpoint_consolidada = pd.read_csv('base_joinpoint_consolidada.csv', header = 0, low_memory=False, index_col=False, sep=';', encoding='latin-1')

In [ ]:
base_espaco_consolidada.dtypes

In [ ]:
base_espaco_tempo_consolidada.dtypes

In [ ]:
base_joinpoint_consolidada.dtypes

In [ ]:
suicidio_intermediario.head(1).to_csv('valida51_campos_atuais.csv')

In [ ]:
suicidio_base_geral = suicidio_intermediario.merge(base_espaco_consolidada, on=['cod_mun'],  how='left')

In [ ]:
suicidio_base_geral = suicidio_base_geral.merge(base_espaco_tempo_consolidada, on=['cod_mun'],  how='left')

In [ ]:
suicidio_base_geral = suicidio_base_geral.merge(base_joinpoint_consolidada, on=['cod_mun'],  how='left')

In [ ]:
suicidio_base_geral.to_csv('suicidio_base_geral.csv')

In [ ]:
#suicidio_intermediario_munexcluidos[['cod_mun', 'MUNNOMEX','ano','taxa_padronizada_sexo_idade']]

In [ ]:
suicidio_base_geral.head(1).to_csv('valida52_campos_totais.csv')

In [ ]:
#exclui algumas colunas

In [ ]:
suicidio_base_geral_filtrado=suicidio_base_geral.loc[:, ~suicidio_base_geral.columns.isin(['sexo_x','municipio_x','cod_ibge_num_x','sexo_y',
                                                              'municipio_y','cod_ibge_num_y','NM_UF','CD_GCMUN',
                                                              'MUNCOD_x','NM_MUN','MUNCOD_y','MUNNOME'])]

In [ ]:
suicidio_base_geral_filtrado.to_csv('suicidio_base_geral_filtrada.csv')

In [ ]:
#faz uma base menor

In [2]:
suicidio_base_geral = pd.read_csv('suicidio_base_geral.csv', header = 0, low_memory=False, index_col=[0], sep=',', encoding='latin-1')

In [ ]:
suicidio_base_geral.head(1).to_csv('valida52_campos.csv')

In [ ]:
suicidio_base_geral_filtrado2=suicidio_base_geral.loc[:, ~suicidio_base_geral.columns.isin(['sexo_x','municipio_x','cod_ibge_num_x','sexo_y',
                                                              'municipio_y','cod_ibge_num_y','NM_UF','CD_GCMUN',
                                                              'propwho_75_anos_e_mais','propwho_65_a_74_anos','propwho_55_a_64_anos','propwho_45_a_54_anos','propwho_35_a_44_anos','propwho_25_a_34_anos','propwho_15_a_24_anos','propwho_5_a_14_anos','propwho_0_a_4_anos',
                                                              'MUNCOD_x','NM_MUN','MUNCOD_y','MUNNOME', 'join_mse_recente','join_apc_significativo_recente','join_data_fim_recente','join_data_inicio_recente','join_amplitude_APC_recente','join_tendencia_4','join_criterio_conservador_significativo_4','join_mse_4','join_apc_significativo_4','join_data_fim_4','join_data_inicio_4','join_amplitude_APC_4','join_apc_4','join_tendencia_3','join_criterio_conservador_significativo_3','join_mse_3','join_apc_significativo_3','join_data_fim_3','join_data_inicio_3','join_amplitude_APC_3','join_apc_3','join_tendencia_2','join_criterio_conservador_significativo_2','join_mse_2','join_apc_significativo_2','join_data_fim_2','join_data_inicio_2','join_amplitude_APC_2','join_apc_2','join_tendencia_1','join_criterio_conservador_significativo_1','join_mse_1','join_apc_significativo_1','join_data_fim_1','join_data_inicio_1','join_amplitude_APC_1','join_apc_1','join_tendencia_0','join_criterio_conservador_significativo_0','join_mse_0','join_apc_significativo_0','join_data_fim_0','join_data_inicio_0','join_amplitude_APC_0','join_apc_0','ESPTEMP_LOC_POP','ESPTEMP_LOC_RR','ESPTEMP_LOC_ODE','ESPTEMP_LOC_EXP','ESPTEMP_LOC_OBS','ESPTEMP_CLU_POP','ESPTEMP_CLU_RR','ESPTEMP_CLU_ODE','ESPTEMP_CLU_EXP','ESPTEMP_CLU_OBS','ESPTEMP_P_VALUE','ESPACIAL_DATA_REF','ESPACIAL_LOC_POP','ESPACIAL_LOC_RR','ESPACIAL_LOC_ODE','ESPACIAL_LOC_EXP','ESPACIAL_LOC_OBS','ESPACIAL_CLU_POP','ESPACIAL_CLU_RR','ESPACIAL_CLU_ODE','ESPACIAL_CLU_EXP','ESPACIAL_CLU_OBS','ESPACIAL_P_VALUE'])]

In [ ]:
suicidio_base_geral_filtrado2.to_csv('suicidio_base_geral_filtrada2.csv')

In [ ]:
analise_ruralidade = suicidio_base_geral[suicidio_base_geral.ano == 2019][['cod_mun','pop_total','ESPACIAL_CLU_RISCO', 'TIPO','ESPACIAL_CLUSTER']]

In [ ]:
analise_ruralidade.to_csv('analise_ruralidade.csv')

In [ ]:
suicidio_base_geral[suicidio_base_geral.index == 22268] .to_csv('valida53_erro_powerbi.csv')

In [ ]:
#tem muita cidade com idade ignorada, lembrar de não considerar nas minhas contas
suicidio_base_geral[suicidio_base_geral.pop_idade_ignorada == 0].groupby(['ano'])['cod_mun'].agg('count')

#df.groupby(['Name', 'Fruit'])['Number'].agg('sum')

In [ ]:
suicidio_base_geral[suicidio_base_geral.pop_idade_ignorada == 0].agg('count')

In [ ]:
#validacao dia 21/04: quero entender mapa do power bi se limita taxa

In [ ]:
#suicidio_base_geral, taxa_padronizada_sexo_idade

In [5]:
suicidio_teste = suicidio_base_geral[(suicidio_base_geral.ano == 2019) & (suicidio_base_geral.taxa_padronizada_sexo_idade>20)][['SIG_UF','MUNNOMEX','taxa_padronizada_sexo_idade']] 

In [6]:
suicidio_teste.head(5)

,SIG_UF,MUNNOMEX,taxa_padronizada_sexo_idade
128035,RO,ALTA FLORESTA D'OESTE,22.420654
128040,RO,COLORADO DO OESTE,20.714307
128055,RO,SANTA LUZIA D'OESTE,34.764018
128070,RO,GOVERNADOR JORGE TEIXEIRA,21.002907
128075,RO,NOVA UNIAO,31.141457


# CÓDIGOS ANTIGOS - DECONSIDERAR

## Monta base para análises 2010

In [ ]:
suicidio_2010 = suicidio[suicidio.ano == 2010][['cod_mun','taxa_padronizada','taxa_padronizada_masc','taxa_padronizada_fem','taxa_padronizada_sexo_idade']] 

In [ ]:
#suicidio_2010.dtypes

In [ ]:
mapa_2010=mapa2.merge(suicidio_2010, on=['cod_mun'], how='left')  

In [ ]:
#mapa_2010.head(5)

In [ ]:
#procura nulo, achou todos... ufa
#mapa_2010[mapa_2010.taxa_padronizada.isnull() ]

### Base para 2010 pronta, agora vamos plotar o mapa

In [ ]:
#https://rodrigodutcosky.medium.com/mapas-coropl%C3%A9ticos-com-os-estados-do-brasil-em-python-b9b48c6db585
%matplotlib inline

mapa_2010.plot(column = 'taxa_padronizada',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

In [ ]:
mapa_2010_outlier=mapa_2010.copy()

In [ ]:
#antes era 20, mudei para 10
mapa_2010_outlier['taxa_padronizada'] = np.where(mapa_2010_outlier.taxa_padronizada>10,0,mapa_2010_outlier.taxa_padronizada)
mapa_2010_outlier['taxa_padronizada_sexo_idade'] = np.where(mapa_2010_outlier.taxa_padronizada_sexo_idade>10,0,mapa_2010_outlier.taxa_padronizada_sexo_idade)

In [ ]:
#começa com 16,10 figsize
%matplotlib inline

mapa_2010_outlier.plot(column = 'taxa_padronizada',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

## Analisando a taxa padronizada por sexo e idade

In [ ]:
#https://rodrigodutcosky.medium.com/mapas-coropl%C3%A9ticos-com-os-estados-do-brasil-em-python-b9b48c6db585
%matplotlib inline

mapa_2010.plot(column = 'taxa_padronizada_sexo_idade',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

In [ ]:
#começa com 16,10 figsize
%matplotlib inline

mapa_2010_outlier.plot(column = 'taxa_padronizada_sexo_idade',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

# Suicídio 2019

In [ ]:
suicidio_2019 = suicidio[suicidio.ano == 2019][['cod_mun','taxa_padronizada','taxa_padronizada_masc','taxa_padronizada_fem','taxa_padronizada_sexo_idade']] 

In [ ]:
mapa_2019=mapa2.merge(suicidio_2019, on=['cod_mun'], how='left')  

In [ ]:
#procura nulo, achou todos... ufa
mapa_2019[mapa_2019.taxa_padronizada.isnull() ]

### Base para 2019 pronta, agora vamos plotar o mapa

In [ ]:
#https://rodrigodutcosky.medium.com/mapas-coropl%C3%A9ticos-com-os-estados-do-brasil-em-python-b9b48c6db585
%matplotlib inline

mapa_2019.plot(column = 'taxa_padronizada',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

In [ ]:
mapa_2019_outlier=mapa_2019.copy()

In [ ]:
mapa_2019_outlier['taxa_padronizada'] = np.where(mapa_2019_outlier.taxa_padronizada>10,0,mapa_2019_outlier.taxa_padronizada)
mapa_2019_outlier['taxa_padronizada_sexo_idade'] = np.where(mapa_2019_outlier.taxa_padronizada_sexo_idade>10,0,mapa_2019_outlier.taxa_padronizada_sexo_idade)

In [ ]:
#começa com 16,10 figsize
%matplotlib inline

mapa_2019_outlier.plot(column = 'taxa_padronizada',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

## Analisando a taxa padronizada por sexo e idade

In [ ]:
#https://rodrigodutcosky.medium.com/mapas-coropl%C3%A9ticos-com-os-estados-do-brasil-em-python-b9b48c6db585
%matplotlib inline

mapa_2019.plot(column = 'taxa_padronizada_sexo_idade',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

In [ ]:
#começa com 16,10 figsize
%matplotlib inline

mapa_2019_outlier.plot(column = 'taxa_padronizada_sexo_idade',
               cmap ='Reds',
               figsize = (50,40),
               legend = True,
               edgecolor = 'grey'
              )

# Analisa outlier

In [ ]:
suicidio = pd.read_csv('base_suicidio_final.csv', header = 0, low_memory=False, index_col=False, sep=',', encoding='latin-1')

In [ ]:
suicidio.dtypes.to_csv('valida23_campos.csv')

In [ ]:
base_reduzida= suicidio[['ano','cod_mun','suic_total', 'pop_total','taxa_padronizada','taxa_padronizada_masc','taxa_padronizada_fem','taxa_padronizada_sexo_idade']]

In [ ]:
base_reduzida.head(5)